In [1]:
import pandas as pd
import os
import numpy as np
import sys
from scipy.interpolate import interp1d

In [2]:
os.chdir('/Users/juansierra/Downloads')

In [3]:
def vf_cte(pago, n, i,valor_inicial):

  pagos = np.ones(n) * pago + np.concatenate(([valor_inicial], np.zeros(n-1)))    
  print(pagos.sum())
  cao_factor = np.ones(n) * (1 + i) ** np.arange(n,0,-1)
  resp_vector = pagos * cao_factor
  resp = resp_vector.sum() 
  return resp

def TMensual(Tmort):
    """
    Interpola los valores de l(x) en pasos de 1/12 (mensual) y calcula q(x) y p(x).
    
    Parameters:
    Tmort (DataFrame): DataFrame que contiene las columnas 'x' y 'l(x)'.
    
    Returns:
    DataFrame: DataFrame con valores interpolados de x, l(x), q(x) y p(x).
    """
    
    # Aseguramos que Tmort tiene las columnas adecuadas
    if 'x' not in Tmort.columns or 'l(x)' not in Tmort.columns:
        raise ValueError("El DataFrame debe contener las columnas 'x' y 'l(x)'")
    
    # Obtener los valores de x y l(x)
    x_values = Tmort['x'].values
    lx_values = Tmort['l(x)'].values
    
    # Crear una función de interpolación
    interp_function = interp1d(x_values, lx_values, kind='linear', fill_value='extrapolate')
    
    # Crear los nuevos valores de x con un paso de 1/12, hasta 110 (incluyendo 110)
    new_x = np.arange(x_values[0], 110 + 1/12, 1/12)  # Incrementos mensuales hasta 110
    
    # Calcular los valores de l(x) interpolados
    new_lx = interp_function(new_x)
    
    # Inicializar los arrays para q(x) y p(x)
    qx_values = np.zeros(len(new_x) - 1)  # Inicializar el array para q(x)
    px_values = np.zeros(len(new_x) - 1)  # Inicializar el array para p(x)

    # Calcular q(x) y p(x) para cada nuevo valor de x
    for i in range(len(new_x)-1):
        if new_lx[i] > 0:  # Evitar división por cero
            qx_values[i] = 1 - (new_lx[i + 1] / new_lx[i])  # q(x) = 1 - l(x + 1/12) / l(x)
        else:
            qx_values[i] = 0  # Si l(x) es 0, q(x) también debe ser 0

        px_values[i] = 1 - qx_values[i]  # p(x) = 1 - q(x)


    # Crear un nuevo DataFrame con los valores interpolados y calculados
    interpolated_df = pd.DataFrame({
        'x': new_x,  # Incluir hasta 110
        'l(x)': new_lx,  # Excluir el último por la misma razón
        'q(x)': np.append(qx_values, 1),  # Añadir el valor de q(110)
        'p(x)': np.append(px_values, 0)  # Añadir el valor de p(110)
    })
    
    return interpolated_df

def tmort_filtered(df,edad):
    qx_vec=df[df['x']>=edad]
    return qx_vec

In [21]:
# Información del Trabajador
edad_actual = 30  # Edad actual del trabajador
genero = 'H'  # Género del trabajador (H: Hombre)

# Parámetros Financieros
tasa_contribucion = 0.04  # Tasa de contribución mensual
salario_base = 7_000_000  # Salario base mensual
edad_jubilacion = 62  # Edad esperada de jubilación

# Parámetros de Inversión
tasa_anual_rendimiento = 0.04  # Tasa anual de rendimiento de inversiones

In [23]:
# Cálculo de Pago Mensual
# Asume una contribución del 16% del salario base
pago_mensual = salario_base * 0.16

# Cálculo del Número de Meses hasta la Jubilación
# Multiplica por 12 para convertir años a meses
numero_meses = (edad_jubilacion - edad_actual) * 12

# Cálculo de la Tasa de Interés Mensual
# Divide la tasa anual por 12 para obtener la tasa mensual
tasa_interes_mensual = tasa_anual_rendimiento / 12

# Valor inicial de inversión (generalmente 0 al comenzar)
valor_inicial = 0

# Calcula el valor futuro de las contribuciones
valor_futuro = vf_cte(pago_mensual, numero_meses, tasa_interes_mensual, valor_inicial)

# Impresión de Resultados
print("Valor Futuro:", valor_futuro)
print("Total Contribuciones:", numero_meses * pago_mensual)

430080000.0
Valor Futuro: 872800998.1122966
Total Contribuciones: 430080000.0


In [24]:
tabla_mortalidad_mujeres = pd.read_excel('TM_mujeres.xlsx')

tabla_mortalidad_hombres = pd.read_excel('TM_hombres.xlsx')

In [25]:
tabla_mortalidad_mensual_hombres = TMensual(tabla_mortalidad_hombres)

tabla_mortalidad_mensual_mujeres = TMensual(tabla_mortalidad_mujeres)

In [26]:
print(tabla_mortalidad_mensual_hombres.head(15))

            x            l(x)      q(x)      p(x)
0   15.000000  1000000.000000  0.000040  0.999960
1   15.083333   999959.583333  0.000040  0.999960
2   15.166667   999919.166667  0.000040  0.999960
3   15.250000   999878.750000  0.000040  0.999960
4   15.333333   999838.333333  0.000040  0.999960
5   15.416667   999797.916667  0.000040  0.999960
6   15.500000   999757.500000  0.000040  0.999960
7   15.583333   999717.083333  0.000040  0.999960
8   15.666667   999676.666667  0.000040  0.999960
9   15.750000   999636.250000  0.000040  0.999960
10  15.833333   999595.833333  0.000040  0.999960
11  15.916667   999555.416667  0.000040  0.999960
12  16.000000   999515.000000  0.000041  0.999959
13  16.083333   999473.666667  0.000041  0.999959
14  16.166667   999432.333333  0.000041  0.999959


In [27]:
tabla_filtrada = tmort_filtered(tabla_mortalidad_mensual_hombres, edad_jubilacion)
print(tabla_filtrada.head(15))

             x           l(x)      q(x)      p(x)
564  62.000000  897019.000000  0.000766  0.999234
565  62.083333  896331.500000  0.000767  0.999233
566  62.166667  895644.000000  0.000768  0.999232
567  62.250000  894956.500000  0.000768  0.999232
568  62.333333  894269.000000  0.000769  0.999231
569  62.416667  893581.500000  0.000769  0.999231
570  62.500000  892894.000000  0.000770  0.999230
571  62.583333  892206.500000  0.000771  0.999229
572  62.666667  891519.000000  0.000771  0.999229
573  62.750000  890831.500000  0.000772  0.999228
574  62.833333  890144.000000  0.000772  0.999228
575  62.916667  889456.500000  0.000773  0.999227
576  63.000000  888769.000000  0.000856  0.999144
577  63.083333  888007.833333  0.000857  0.999143
578  63.166667  887246.666667  0.000858  0.999142


In [28]:
tabla_filtrada['n'] = tabla_filtrada['x'].apply(lambda x: (x - edad_jubilacion) * 12)
print(tabla_filtrada)

               x           l(x)      q(x)      p(x)             n
564    62.000000  897019.000000  0.000766  0.999234  4.007461e-12
565    62.083333  896331.500000  0.000767  0.999233  1.000000e+00
566    62.166667  895644.000000  0.000768  0.999232  2.000000e+00
567    62.250000  894956.500000  0.000768  0.999232  3.000000e+00
568    62.333333  894269.000000  0.000769  0.999231  4.000000e+00
...          ...            ...       ...       ...           ...
1136  109.666667     121.333333  0.083104  0.916896  5.720000e+02
1137  109.750000     111.250000  0.090637  0.909363  5.730000e+02
1138  109.833333     101.166667  0.099671  0.900329  5.740000e+02
1139  109.916667      91.083333  0.110704  0.889296  5.750000e+02
1140  110.000000      81.000000  1.000000  0.000000  5.760000e+02

[577 rows x 5 columns]


/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/1394774574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_filtrada['n'] = tabla_filtrada['x'].apply(lambda x: (x - edad_jubilacion) * 12)


In [29]:
# 1/(1+i) es el factor de descuento por período
tabla_filtrada['Vn'] = np.ones(len(tabla_filtrada)) * (1 / (1 + tasa_interes_mensual))

# Calcular el factor de descuento acumulado
# Eleva el factor base a la potencia de 'n' (meses)
tabla_filtrada['Vn'] = tabla_filtrada.apply(lambda x: x['Vn'] ** x['n'], axis=1)

print(tabla_filtrada)

               x           l(x)      q(x)      p(x)             n        Vn
564    62.000000  897019.000000  0.000766  0.999234  4.007461e-12  1.000000
565    62.083333  896331.500000  0.000767  0.999233  1.000000e+00  0.996678
566    62.166667  895644.000000  0.000768  0.999232  2.000000e+00  0.993367
567    62.250000  894956.500000  0.000768  0.999232  3.000000e+00  0.990066
568    62.333333  894269.000000  0.000769  0.999231  4.000000e+00  0.986777
...          ...            ...       ...       ...           ...       ...
1136  109.666667     121.333333  0.083104  0.916896  5.720000e+02  0.149047
1137  109.750000     111.250000  0.090637  0.909363  5.730000e+02  0.148551
1138  109.833333     101.166667  0.099671  0.900329  5.740000e+02  0.148058
1139  109.916667      91.083333  0.110704  0.889296  5.750000e+02  0.147566
1140  110.000000      81.000000  1.000000  0.000000  5.760000e+02  0.147076

[577 rows x 6 columns]


/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/2506411797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_filtrada['Vn'] = np.ones(len(tabla_filtrada)) * (1 / (1 + tasa_interes_mensual))
/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/2506411797.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_filtrada['Vn'] = tabla_filtrada.apply(lambda x: x['Vn'] ** x['n'], axis=1)


In [30]:
# Multiplica la probabilidad de supervivencia 'p(x)' por el factor de descuento 'Vn'
suma_ponderada_vector = tabla_filtrada['p(x)'] * tabla_filtrada['Vn']

# Calcular la suma total de este vector
suma_total = suma_ponderada_vector.sum()

# Convertir a float para asegurar precisión numérica
suma_total = float(suma_total)

print("Suma total ponderada:", suma_total)

Suma total ponderada: 254.5545084058316


In [33]:
resultado_vf_suma = valor_futuro / suma_total
print(resultado_vf_suma)

3428739.1080923458


In [34]:
254/12

21.166666666666668

### Ejercicio mujeres

In [36]:
# Parámetros iniciales
edad_actual = 30 
edad_jubilacion = 57 
salario_base = 7000000  
tasa_anual = 0.04  # Tasa anual de rendimiento

# Cálculo de Pago Mensual
pago_mensual = salario_base * 0.16

# Cálculo de Número de Meses hasta Jubilación
n = (edad_jubilacion - edad_actual) * 12

# Tasa de Interés Mensual
i = tasa_anual / 12

# Valor Inicial
valor_inicial = 0

# Cálculo de Valor Futuro de Contribuciones
vf = vf_cte(pago_mensual, n, i, valor_inicial)
print("Valor Futuro de Contribuciones:", vf)

# Cálculo Total de Contribuciones
total_contribuciones = n * pago_mensual
print("Total de Contribuciones:", total_contribuciones)

# Filtrar Tabla de Mortalidad para Mujeres
T = tmort_filtered(tabla_mortalidad_mensual_mujeres, edad_jubilacion)

# Calcular Meses desde Jubilación
T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)

# Calcular Factor de Descuento
T['Vn'] = np.ones(len(T)) * (1 / (1 + i))
T['Vn'] = T.apply(lambda x: x['Vn'] ** x['n'], axis=1)

# Calculo Suma Ponderada de Probabilidades
sum_vector = T['p(x)'] * T['Vn']
suma_total = sum_vector.sum()
print("Suma Total:", suma_total)

362880000.0
Valor Futuro de Contribuciones: 653809052.6404626
Total de Contribuciones: 362880000.0
Suma Total: 263.2164186769155


/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/4033712413.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/4033712413.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn'] = np.ones(len(T)) * (1 / (1 + i))
/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/4033712413.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [37]:
relacion_vf_suma = vf / suma_total
print(relacion_vf_suma)

2483921.998205512


In [38]:
263/12

21.916666666666668

### Cambio de monto inicial

In [40]:
# Parámetros iniciales
edad_actual = 30  # Edad actual
edad_jubilacion = 57  # Edad de jubilación para mujer
salario_base = 7_000_000  # Salario base
tasa_anual = 0.04  # Tasa anual de rendimiento

# Cálculo de Pago Mensual
# Contribución del 16% del salario
pago_mensual = salario_base * 0.16

# Cálculo de Número de Meses hasta Jubilación
n = (edad_jubilacion - edad_actual) * 12

# Tasa de Interés Mensual
i = tasa_anual / 12

# Valor Inicial
valor_inicial = 96000000

# Cálculo de Valor Futuro de Contribuciones Constantes
vf = vf_cte(pago_mensual, n, i, valor_inicial)
print("Valor Futuro de Contribuciones:", vf)

# Cálculo Total de Contribuciones
total_contribuciones = n * pago_mensual
print("Total de Contribuciones:", total_contribuciones)

# Filtrar Tabla de Mortalidad para Mujeres
T = tmort_filtered(tabla_mortalidad_mensual_mujeres, edad_jubilacion)

# Calcular Meses desde Jubilación
T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)

# Calcular Factor de Descuento
T['Vn'] = np.ones(len(T)) * (1 / (1 + i))
T['Vn'] = T.apply(lambda x: x['Vn'] ** x['n'], axis=1)

# Calcular Suma Ponderada de Probabilidades
sum_vector = T['p(x)'] * T['Vn']
suma_total = sum_vector.sum()
print("Suma Total:", suma_total)

458880000.0
Valor Futuro de Contribuciones: 935991032.5096068
Total de Contribuciones: 362880000.0
Suma Total: 263.2164186769155


/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/345067203.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['n'] = T['x'].apply(lambda x: (x - edad_jubilacion) * 12)
/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/345067203.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  T['Vn'] = np.ones(len(T)) * (1 / (1 + i))
/var/folders/jn/6s4fnp_x27qcfq657s92n39m0000gn/T/ipykernel_1247/345067203.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [41]:
resultado_final = vf / suma_total
print(resultado_final)

3555975.1067751106
